# Chapter 3 Solving Non-invertible Linear System

Let us solve $A x = b$ when $A$ is non-square or not of full rank. We will use $LU$-decomposition.

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [5]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0]:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ', rank_u)
    print('Pivot variables = ', np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables = ", free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    print(np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b))))    

Let us test the function solving $A\mathbf{x} = \mathbf{b}$ even when $A$ is not invertible.

In [6]:
# Setting solvability
solvable = 1    # 1 for solvable, 0 for unsolvable
if solvable:
    print("The linear system is solvable.")
else:
    print("The linear system is unsolvable.")
# Setting parameters
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)
if solvable:
    b = A @ np.random.randn(n,1)
else:
    b = np.random.randn(m,1)
    
solve_Ax_b(A,b)

The linear system is solvable.
[[ 5.7619  6.3647  7.1414  1.2009  1.7278 -9.8372  0.6232 -7.1095 -0.1903 -2.8484  4.3657 -5.1335]
 [ 0.     -9.2234 -3.8293 -2.0108  2.0324  2.8078 -0.3201  7.1127 -4.9224  2.5851 -3.4324 -7.599 ]
 [ 0.      0.     -5.2975  0.1061 -5.6942  1.1634 -2.466  -1.8678  5.811   1.1736 -0.1784 10.1115]
 [ 0.      0.      0.     -1.4016 -0.7618 -1.0427 -0.1979 -0.5556  1.2851 -2.605  -0.0066  0.5662]
 [ 0.      0.      0.      0.      2.3941 -1.8033  3.0678 -2.5485  0.8743  3.4429  4.2455  4.0386]
 [ 0.      0.      0.      0.      0.     -1.7707  2.5419  0.912  -2.1566 -4.098   0.9866  0.4287]
 [ 0.      0.      0.      0.      0.      0.     -1.6895 -1.5572  3.489  -3.2183 -1.3221  0.7518]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0. 